In [1]:
import pandas as pd

In [2]:
import ast #usamos para convertir valores string en columna en listas reales

In [3]:
import re

In [4]:
data = pd.read_csv("df_corregido.csv")

In [5]:
df = data.copy()

In [6]:
df

,nom,ingredients,passos,desdejuni,dinar,sopar,snack,apartat
0,Iogurt grec amb fraules,"['Iogurt grec 200g', 'Fraules 100-150g', 'Nous...",['Tallar les fraules i mesclar tot en un bol.'...,1,0,1,1,[]
1,Solomillo de vedella Strogonoff i arròs basmati,"['Solomillo de vedella 150-200g', 'Xampinyons ...","['Adobar la carn amb sal i prebe bò.', 'Posar ...",0,1,0,0,[]
2,Espàrrecs embolicats en pernil,"['½ manat d’espàrrecs', '3-4 llesques de perni...","['Preparar forn 150ºC.', 'Xepam les llesques d...",1,0,1,0,[]
3,Mini amanida verda,"[""20g d'enciam roman"", '15g de ruca', ""20g d'e...",['Mesclar tots els ingredinets en un bol + ½ a...,1,1,1,1,['Amanida']
4,Cuinat de llenties,"['150g de llenties crues', ""750ml d'aigua"", ""2...",['Posar les llenties prèviament remullades i c...,1,1,0,0,[]
...,...,...,...,...,...,...,...,...
143,Pebrots farcits de peix i formatge.,['400g de peix (si ho fas amb rap queda molt b...,"['Encalentir el forn a 180°C.', 'Rentar i tall...",0,1,0,0,[]
144,Ou bullit amb salmó fumat i ½ alvocat,"['2 OU', '100 g de salmó fumat', '½ ALVOCAT', ...",['Bullir l’ou durante 8 minuts i quan estigui ...,1,0,1,0,[]
145,Ous remenats o fregits amb cuixot salat,"['3 ous', '35 g de pernil', ""8 ml d'oli d'oliv...","['Encalentir l’oli a la paella, tirar els ous,...",1,0,1,0,[]
146,Espàrrecs saltejats,"['200g d’espàrrecs', '10 g de mantega', 'pebre...","['Rentar i tallar els espàrrecs', 'Encalentir ...",0,1,1,0,[]


### Conversión de valores strings en cadenas reales para facilitar manipulacion

In [7]:
df["ingredients"] = df["ingredients"].apply(ast.literal_eval)

In [8]:
df["passos"] = df["passos"].apply(ast.literal_eval)

In [9]:
df["apartat"] = df["apartat"].apply(ast.literal_eval)

### Conseguir "apartat" de cada receta

In [10]:
# Crea "apartat" si no existe

if "apartat" not in df.columns:
    df["apartat"] = [[] for _ in range(len(df))]

In [11]:
# Introduce Amanida 

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Amanida"])) if "amanida" in row["nom"].lower() else row["apartat"],
    axis=1
)

In [12]:
# Introduce Carn Blanca 

#carn_blanca = ["pollastre", "gall dindi", "gall d'indi", "conill", "guatlla", "ànec", "gallina", "faisà", "colomí"]

#df["apartat"] = df.apply(
#    lambda row: list(set(row["apartat"] + ["Carn blanca"])) 
#    if any(ingredient in row["nom"].lower() for ingredient in carn_blanca) or any(ingredient in ing.lower() for ingredient in carn_blanca for ing in row["ingredients"]) 
#    else row["apartat"],
#    axis=1
#)

'''
Tenemos el problema de "brou de pollastre", así que tenemos que añadir una función que excluya esta coincidencia
'''

carn_blanca = ["pollastre", "gall dindi", "gall d'indi", "conill", "guatlla", "ànec", "gallina", "faisà", "colomí"]
exclusiones = ["brou de pollastre", "brou casolà de pollastre"]

def es_carne_blanca_valida(ingrediente):
    ingrediente_lower = ingrediente.lower()
    if any(excl in ingrediente_lower for excl in exclusiones):
        return False
    return any(carne in ingrediente_lower for carne in carn_blanca)

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Carn blanca"]))
    if any(ingredient in row["nom"].lower() for ingredient in carn_blanca) or
       any(es_carne_blanca_valida(ing) for ing in row["ingredients"])
    else row["apartat"],
    axis=1
)

In [13]:
# Introduce Carn Vermella

carn_vermella = ["vedella", "vedell", "bou", "vaca", "cabrit", "porc", "cavall", "búfal", "xoriç", "pernil", "carn picada"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Carn vermella"])) 
    if any(ingredient in row["nom"].lower() for ingredient in carn_vermella) or any(ingredient in ing.lower() for ingredient in carn_vermella for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [14]:
# Introduce Pescado

peix = ["salmó", "bacallà", "tonyina", "dorada", "peix", "lubina", "truita"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Peix"])) 
    if any(ingredient in row["nom"].lower() for ingredient in peix) or any(ingredient in ing.lower() for ingredient in peix for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [15]:
# Introduce Postre

postre = ["xocolata", "bombó", "muffins", "fraules", "iogurt grec amb", "cetogranola", "granola"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Postre"])) 
    if any(ingredient in row["nom"].lower() for ingredient in postre) or any(ingredient in ing.lower() for ingredient in postre for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [16]:
# Introduce Marisco

marisc = ["gambes", "llagostins", "clöisses", "musclos"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Marisc"])) 
    if any(ingredient in row["nom"].lower() for ingredient in marisc) or any(ingredient in ing.lower() for ingredient in marisc for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [17]:
# Introduce Crema 

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Crema"])) if "crema" in row["nom"].lower() else row["apartat"],
    axis=1
)

In [18]:
# Introduce Iogurt 

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Iogurt"])) if "iogurt" in row["nom"].lower() else row["apartat"],
    axis=1
)

In [19]:
# Introduce Ous
'''
Hubo problemas ya que detectaba ou dentro de nous y brous, así que usaremos regex
'''

ous = ["ous", "ou"]

pattern = r'\b(?:' + '|'.join(ous) + r')\b'

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Ou"])) 
    if (
        row["nom"] and isinstance(row["nom"], str) and re.search(pattern, row["nom"].lower())
    ) or (
        row["ingredients"] and isinstance(row["ingredients"], list) and any(
            re.search(pattern, ing.lower()) for ing in row["ingredients"] if isinstance(ing, str)
        )
    )
    else row["apartat"],
    axis=1
)

In [20]:
# Introduce Fruit Secs

f_secs = ["ametlles", "nous", "pinyons", "avellanes", "cacauets", "pipes"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Fruit secs"])) 
    if any(ingredient in row["nom"].lower() for ingredient in f_secs) or any(ingredient in ing.lower() for ingredient in f_secs for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [21]:
# Introduce Ceto

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Ceto"])) if "ceto" in row["nom"].lower() or any("ceto" in ing.lower() for ing in row["ingredients"]) else row["apartat"],
    axis=1
)

In [22]:
# Introduce Guarnició

guarnicio = ["bledes saltejades", "espinacs saltejats", "coliflor al forn", "verduretes al forn", "bròquil al vapor", "espàrrecs saltejats", "cols de brussel·les amb pinyons", 
             "boniato al forn", "patata prebiòtica", "amanida", "ruca trempada", "canonges", "xampinyons saltejats", "bolets saltejats", "pasta de carabassí", "arròs de coliflor", "bolets", "guacamole", "verdures",
            "cuinat de llenties", "boniato rostit", "crocant de formatge", ]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Guarnicio"])) 
    if any(ingredient in row["nom"].lower() for ingredient in guarnicio) or any(ingredient in ing.lower() for ingredient in guarnicio for ing in row["ingredients"]) 
    else row["apartat"],
    axis=1
)

In [23]:
# Introduce Salsa

salsa = ["vinagreta", "marinada bbq", "pesto"]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Salsa"])) 
    if any(ingredient in row["nom"].lower() for ingredient in salsa)
    else row["apartat"],
    axis=1
)

In [24]:
#Introduce Pseudopasta
#Incluye exclusion de "pastanaga"

pseudopasta = ["tallarí", "tallarins", "espagueti", "espaguetis", "lasanya", "lasaña","risotto", "rissotto", "pizza", "macarró", "macarrons", "canelons",
                "fetuccini", "fettuccine", "pasta", "pad-thai"]
exclusion = ["pastanaga"]

def es_pseudopasta_valida(row):
    nombre_lower = row["nom"].lower()
    if any(excl in nombre_lower for excl in exclusion):
        return False
    return any(pasta in nombre_lower for pasta in pseudopasta)

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Pseudopasta"])) 
    if es_pseudopasta_valida(row)
    else row["apartat"],
    axis=1
)

In [25]:
# Introduce Pa

pa = ["pa de", "pa ("]

df["apartat"] = df.apply(
    lambda row: list(set(row["apartat"] + ["Pa"])) 
    if any(ingredient in row["nom"].lower() for ingredient in pa)
    else row["apartat"],
    axis=1
)

In [26]:
df.head()

,nom,ingredients,passos,desdejuni,dinar,sopar,snack,apartat
0,Iogurt grec amb fraules,"[Iogurt grec 200g, Fraules 100-150g, Nous i am...","[Tallar les fraules i mesclar tot en un bol., ...",1,0,1,1,"[Postre, Fruit secs, Iogurt]"
1,Solomillo de vedella Strogonoff i arròs basmati,"[Solomillo de vedella 150-200g, Xampinyons 100...","[Adobar la carn amb sal i prebe bò., Posar la ...",0,1,0,0,"[Fruit secs, Carn vermella]"
2,Espàrrecs embolicats en pernil,"[½ manat d’espàrrecs, 3-4 llesques de pernil s...","[Preparar forn 150ºC., Xepam les llesques de c...",1,0,1,0,[Carn vermella]
3,Mini amanida verda,"[20g d'enciam roman, 15g de ruca, 20g d'espina...",[Mesclar tots els ingredinets en un bol + ½ al...,1,1,1,1,"[Amanida, Guarnicio]"
4,Cuinat de llenties,"[150g de llenties crues, 750ml d'aigua, 2 dent...",[Posar les llenties prèviament remullades i co...,1,1,0,0,[Guarnicio]


In [27]:
df.iloc[53]

nom                                                 Cetp-nutella
ingredients    [40g de xocolata 90%, 30g d'avellanes naturals...
passos         [Col·locar les avellanes a la picadora. Picar ...
desdejuni                                                      1
dinar                                                          0
sopar                                                          0
snack                                                          1
apartat                                     [Postre, Fruit secs]
Name: 53, dtype: object

In [28]:
df["nom"].replace("Cetp-nutella", "Ceto-nutella", inplace=True)

C:\Users\antdo\AppData\Local\Temp\ipykernel_30112\331454593.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["nom"].replace("Cetp-nutella", "Ceto-nutella", inplace=True)


### Export DF LIMPIO

In [29]:
df.to_csv("recetas_cat_0301.csv", index=False)

In [48]:
df_filtrado = df[df['apartat'].apply(lambda x: isinstance(x, list) and "Postre" in x)]

In [49]:
df_filtrado

,nom,ingredients,passos,desdejuni,dinar,sopar,snack,apartat
0,Iogurt grec amb fraules,"[Iogurt grec 200g, Fraules 100-150g, Nous i am...","[Tallar les fraules i mesclar tot en un bol., ...",1,0,1,1,"[Postre, Fruit secs, Iogurt]"
24,Pa de plàtan amb xispes de xocolata,"[1 plàtan madur, 1 ou, 15 g de mantega sense s...","[Encalentir el forn a 200°C., Preparar el motl...",1,0,0,1,"[Postre, Ou, Fruit secs, Pa]"
33,Granola,"[80g d'ametlles naturals, 70g de nous naturals...",[Encalentir el forn a 200°C i preparar una saf...,1,0,0,1,"[Postre, Ou, Fruit secs]"
40,Bombó de xocolata,"[40g de xocolata 90%, 40g de llavors de carbas...","[Posar paper de forn damunt la safata., Fondre...",0,0,0,1,[Postre]
53,Ceto-nutella,"[40g de xocolata 90%, 30g d'avellanes naturals...",[Col·locar les avellanes a la picadora. Picar ...,1,0,0,1,"[Postre, Fruit secs]"
61,Muffins de nabius (sense sucre),"[100g de farina d'ametlles, 40g d'eritritol, 2...",[Encalentir el forn a 200°C. Untar els motlles...,1,0,0,1,"[Postre, Ou, Fruit secs]"
63,Coquitos de xocolata,"[40g de coco ratllat, 1 blanc d’ou, 20g d'erit...",[Encalentir el forn a 180°C i preparar la safa...,0,0,0,1,"[Postre, Ou]"
75,Brioix de xocolata,"[40g de xocolata 90%, 1 ou, 2ml d'essència de ...",[Encalentir el forn a 170°C i preparar el motl...,1,0,0,1,"[Postre, Ou, Fruit secs]"
87,Iogurt grec amb nabius i cetogranola,"[150g iogurt grec, 100g nabius, 30g cetogranola]","[Abocar el iogurt, els nabius i la cetogranola...",1,0,1,0,"[Postre, Iogurt, Ceto]"
88,Cetogranola,"[80g d'ametlles naturals, 70g de nous naturals...",[Encalentir el forn a 200°C i preparar una saf...,1,0,0,1,"[Postre, Ou, Fruit secs, Ceto]"


### Exploración "apartat"

In [38]:
apartat_total = []

for apartat in df["apartat"]:
    apartat_total.extend(apartat)

In [40]:
apartats_unics = set(apartat_total)

In [43]:
apartat_count = {}

for apartat in apartats_unics:
    apartat_count[apartat] = 0

In [46]:
for apartat_list in df["apartat"]:
    for apartat in apartat_list:
        if apartat in apartat_count:
            apartat_count[apartat] += 1
        else:
            apartat_count[apartat] = 1

In [47]:
apartat_count

{'Postre': 14,
 'Amanida': 18,
 'Ou': 46,
 'Carn blanca': 19,
 'Iogurt': 7,
 'Guarnicio': 50,
 'Pa': 5,
 'Pseudopasta': 8,
 'Peix': 33,
 'Crema': 7,
 'Marisc': 10,
 'Pollastre': 13,
 'Carn vermella': 37,
 'Salsa': 5,
 'Fruit secs': 36,
 'Ceto': 6}